In [37]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [ ]:
class FanVoteEstimator:
    def __init__(
        self,
        method="rank",
        lambda_constraint=1000.0,
        lambda_smooth=1.0,
        epsilon=1e-6,
        use_features=False,
        feature_names=None,
    ):
        self.method = method
        self.lambda_constraint = lambda_constraint
        self.lambda_smooth = lambda_smooth
        self.epsilon = epsilon
        self.use_features = use_features  # 是否使用特征
        self.feature_names = feature_names  # 特征名称列表

        # 将在fit中初始化
        self.N = None  # 选手数
        self.T = None  # 周数
        self.params = None  # 参数向量
        self.X = None  # 评委分数矩阵
        self.e = None  # 淘汰顺序
        self.elimination_weeks = None  # 淘汰周列表
        self.F = None  # 特征矩阵 [N x D]，D是特征维度

    def fit(self, X, e, elimination_weeks=None, features=None, max_iter=1000, tol=1e-6):
        """
        features: 特征矩阵 [N x D]，每行对应一个选手的特征向量，包括：年龄、行业编码、地区编码等
        """
        self.X = X
        self.e = e

        if elimination_weeks is None:
            self.elimination_weeks = list(range(len(e)))
        else:
            self.elimination_weeks = elimination_weeks

        self.N, self.T = X.shape

        # 存储特征矩阵
        if features is not None and self.use_features:
            self.F = features
            D = features.shape[1]  # 特征维度
        else:
            self.F = None
            D = 0

        # 参数向量结构：
        # 如果使用特征：θ = [w (D维), β (1维), γ_1, ..., γ_T (T维)]
        # 如果不使用特征：θ = [α_1, ..., α_N (N维), β (1维), γ_1, ..., γ_T (T维)]

        if self.use_features and self.F is not None:
            total_params = D + 1 + self.T
        else:
            total_params = self.N + 1 + self.T

        # 初始化参数
        np.random.seed(42)
        initial_params = np.random.randn(total_params) * 0.1

        # 使用SciPy优化
        result = minimize(
            self.objective_function,
            initial_params,
            method="L-BFGS-B",
            options={"maxiter": max_iter, "ftol": tol, "gtol": tol, "disp": False},
        )

        self.params = result.x
        self.fan_votes_ = self._compute_fan_votes(self.params)

        return self

    def _compute_fan_votes(self, params):
        """从参数计算粉丝投票矩阵，使用 log(V) = α_i + βX + γ_t 公式"""
        N, T = self.N, self.T

        if self.use_features and self.F is not None:
            # 使用特征线性组合计算 α_i = w^T * F_i
            D = self.F.shape[1]
            w = params[:D]
            beta = params[D]
            gamma = params[D + 1 :]

            # 计算 α_i 向量
            alpha = self.F @ w  # [N x 1]
        else:
            # 原来的独立参数
            alpha = params[:N]
            beta = params[N]
            gamma = params[N + 1 :]

        # 计算 log(V)
        log_V = alpha[:, np.newaxis] + beta * self.X + gamma[np.newaxis, :]

        # 确保数值稳定性
        log_V = np.clip(log_V, -20, 20)
        V = np.exp(log_V)

        return V

    def _active_contestants(self, week):
        """返回第week周仍在比赛的选手索引列表"""
        if week == 0:
            return list(range(self.N))
        else:
            # 找出在week周之前被淘汰的选手
            eliminated_before = []
            for i, elim_week in enumerate(self.elimination_weeks):
                if elim_week < week:
                    eliminated_before.append(self.e[i])
            return [i for i in range(self.N) if i not in eliminated_before]

    def _constraint_violation(self, week, V):
        """计算第week周的约束违反量"""
        # 如果这周没有淘汰，返回0
        if week not in self.elimination_weeks:
            return 0.0

        # 找出这周的淘汰者在elimination_weeks中的位置
        elim_idx = self.elimination_weeks.index(week)
        e_t = self.e[elim_idx]  # 这周被淘汰的选手

        active = self._active_contestants(week)
        if e_t not in active:
            # 淘汰者不在活跃列表中，说明已经处理过
            return 0.0

        e_idx_in_active = active.index(e_t)
        week_X = self.X[active, week]
        week_V = V[active, week]

        if self.method == "rank":
            # 计算排名
            X_rank = self._compute_rank(-week_X)
            V_rank = self._compute_rank(-week_V)
            combined = X_rank + V_rank

            # 淘汰者应有最大综合排名
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                max_others = np.max(combined_others)
            else:
                max_others = combined[e_idx_in_active]

            violation = max_others - combined[e_idx_in_active] + self.epsilon

        else:  # 'percent'
            # 计算百分比
            X_percent = week_X / week_X.sum()
            V_percent = week_V / week_V.sum()
            combined = X_percent + V_percent

            # 淘汰者应有最小综合百分比
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                min_others = np.min(combined_others)
            else:
                min_others = combined[e_idx_in_active]

            violation = combined[e_idx_in_active] - min_others + self.epsilon

        return max(0, violation)

    def _compute_rank(self, scores):
        """计算排名"""
        order = np.argsort(scores)
        ranks = np.empty_like(order)
        ranks[order] = np.arange(len(scores))

        # 处理并列
        unique_scores, inverse, counts = np.unique(
            scores, return_inverse=True, return_counts=True
        )

        for i in range(len(unique_scores)):
            if counts[i] > 1:
                indices = np.where(inverse == i)[0]
                avg_rank = ranks[indices].mean()
                ranks[indices] = avg_rank

        return ranks + 1
    def objective_function(self, params):
        """目标函数"""
        V = self._compute_fan_votes(params)

        # 1. 约束惩罚项
        constraint_penalty = 0.0
        for week in self.elimination_weeks:
            violation = self._constraint_violation(week, V)
            if violation > 0:
                constraint_penalty += violation**2

        # 2. 平滑项
        smooth_penalty = np.sum((V[:, 1:] - V[:, :-1]) ** 2)

        # 3. 总目标
        total = (
            self.lambda_constraint * constraint_penalty
            + self.lambda_smooth * smooth_penalty
        )

        return total

    def predict(self, X=None):
        """返回估计的粉丝投票"""
        if X is None:
            X = self.X
        return self.fan_votes_
    
    def _constraint_violation(self, week, V):
        """计算第week周的约束违反量，确保非负"""
        # 如果这周没有淘汰，返回0
        if week not in self.elimination_weeks:
            return 0.0
        
        # 找出这周的淘汰者
        elim_idx = self.elimination_weeks.index(week)
        e_t = self.e[elim_idx]
        
        active = self._active_contestants(week)
        if e_t not in active:
            return 0.0
        
        e_idx_in_active = active.index(e_t)
        week_X = self.X[active, week]
        week_V = V[active, week]
        
        if self.method == 'rank':
            # 计算排名
            X_rank = self._compute_rank(-week_X)
            V_rank = self._compute_rank(-week_V)
            combined = X_rank + V_rank
            
            # 淘汰者应有最大综合排名
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                max_others = np.max(combined_others)
            else:
                max_others = combined[e_idx_in_active]
            
            violation = max_others - combined[e_idx_in_active] + self.epsilon
        
        else:  # 'percent'
            # 计算百分比
            X_percent = week_X / week_X.sum()
            V_percent = week_V / week_V.sum()
            combined = X_percent + V_percent
            
            # 淘汰者应有最小综合百分比
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                min_others = np.min(combined_others)
            else:
                min_others = combined[e_idx_in_active]
            
            violation = combined[e_idx_in_active] - min_others + self.epsilon
        
        # 确保非负
        return max(0.0, violation)
    
    def set_voting_method(self, method):
        """设置投票方法: 'rank' 或 'percent'"""
        if method not in ['rank', 'percent']:
            raise ValueError("method must be 'rank' or 'percent'")
        self.method = method

In [39]:
def prepare_season_data_with_features(season_num, raw_data, use_features=True):
    """
    为指定赛季准备数据，包括特征
    返回: X, e, names, elimination_weeks, features
    """
    # 筛选指定赛季的数据
    season_data = raw_data[raw_data["season"] == season_num].copy()

    # 重置索引
    season_data = season_data.reset_index(drop=True)

    # 提取选手名称
    contestant_names = season_data["celebrity_name"].tolist()
    N = len(contestant_names)

    # 确定最大周数
    max_week = 0
    for col in season_data.columns:
        if "week" in col and "judge" in col:
            week_num = int(col.split("_")[0].replace("week", ""))
            max_week = max(max_week, week_num)

    # 初始化评委分数矩阵
    X = np.zeros((N, max_week))

    # 填充评委分数矩阵
    for i in range(N):
        for week in range(1, max_week + 1):
            week_cols = [col for col in season_data.columns if f"week{week}_" in col]
            week_total = 0
            judge_count = 0

            for col in week_cols:
                score = season_data.at[i, col]
                if pd.isna(score) or score == "N/A" or score == 0:
                    continue
                try:
                    score_val = float(score)
                    week_total += score_val
                    judge_count += 1
                except:
                    continue

            if judge_count > 0:
                X[i, week - 1] = week_total / judge_count

    # 解析淘汰信息（同之前）
    # ... [保持原来的淘汰信息解析代码不变] ...

    # 特征工程
    features = None
    if use_features:
        features = extract_features(season_data)

    # 确定实际需要处理的周数
    if len(elimination_weeks) > 0:
        T = max(elimination_weeks) + 1
    else:
        T = max_week

    # 截断X矩阵到T周
    X = X[:, :T]

    return X, e, contestant_names, elimination_weeks, features


def extract_features(data):
    """
    从原始数据中提取特征
    返回特征矩阵 [N x D]
    """
    N = len(data)
    features_list = []

    for i in range(N):
        feature_vector = []

        # 1. 年龄特征
        age = data.at[i, "celebrity_age_during_season"]
        if pd.isna(age):
            age = 40  # 默认值
        feature_vector.append(float(age))

        # 2. 行业特征（独热编码）
        industry = str(data.at[i, "celebrity_industry"])
        industries = [
            "Actor/Actress",
            "Athlete",
            "Singer/Rapper",
            "Model",
            "TV Personality",
            "News Anchor",
            "Politician",
            "Other",
        ]

        industry_encoded = [0] * len(industries)
        if industry in industries:
            industry_encoded[industries.index(industry)] = 1
        else:
            industry_encoded[-1] = 1  # 'Other'类别
        feature_vector.extend(industry_encoded)

        # 3. 地区特征（简化：美国/非美国）
        country = str(data.at[i, "celebrity_homecountry/region"])
        is_usa = 1 if "United States" in country else 0
        feature_vector.append(is_usa)

        # 4. 是否为体育明星（额外特征）
        is_athlete = 1 if "Athlete" in industry else 0
        feature_vector.append(is_athlete)

        features_list.append(feature_vector)

    features = np.array(features_list)

    # 对数值特征进行标准化
    numeric_cols = [0]  # 年龄列
    for col in numeric_cols:
        mean_val = np.mean(features[:, col])
        std_val = np.std(features[:, col])
        if std_val > 0:
            features[:, col] = (features[:, col] - mean_val) / std_val

    return features

In [40]:
def prepare_season_data(season_num, raw_data):
    """
    为指定赛季准备数据
    返回: X(评委分数矩阵), e(淘汰顺序), contestant_names(选手名称), elimination_weeks(淘汰周列表)
    """
    # 筛选指定赛季的数据
    season_data = raw_data[raw_data["season"] == season_num].copy()

    # 重置索引
    season_data = season_data.reset_index(drop=True)

    # 提取选手名称
    contestant_names = season_data["celebrity_name"].tolist()
    N = len(contestant_names)

    # 确定最大周数（从列名中提取）
    max_week = 0
    for col in season_data.columns:
        if "week" in col and "judge" in col:
            week_num = int(col.split("_")[0].replace("week", ""))
            max_week = max(max_week, week_num)

    # 初始化评委分数矩阵
    X = np.zeros((N, max_week))

    # 填充评委分数矩阵
    for i in range(N):
        for week in range(1, max_week + 1):
            # 获取本周所有评委分数列
            week_cols = [col for col in season_data.columns if f"week{week}_" in col]
            week_total = 0
            judge_count = 0

            for col in week_cols:
                score = season_data.at[i, col]
                # 处理N/A和0值
                if pd.isna(score) or score == "N/A" or score == 0:
                    continue
                try:
                    score_val = float(score)
                    week_total += score_val
                    judge_count += 1
                except:
                    continue

            # 如果本周有评委分数，则记录平均值
            if judge_count > 0:
                X[i, week - 1] = week_total / judge_count

    # 解析淘汰信息
    elimination_info = []

    for i in range(N):
        result = str(season_data.at[i, "results"])

        # 检查是否是决赛选手
        if any(place in result for place in ["1st", "2nd", "3rd", "4th", "5th"]):
            # 决赛选手：在最后一周被"淘汰"（实际上不被淘汰，但为了模型需要）
            elim_week = max_week - 1  # 0-based索引
        elif "Withdrew" in result:
            # 退赛情况：找到第一个0分的周次
            elim_week = max_week - 1  # 默认设为最后一周
            for week in range(max_week):
                if X[i, week] == 0 and week > 0:  # 跳过第0周，因为可能本来就是0分
                    elim_week = week - 1  # 退赛前一周
                    break
        elif "Eliminated Week" in result:
            # 解析淘汰周次
            try:
                week_str = result.replace("Eliminated Week", "").strip()
                elim_week = int(week_str) - 1  # 转换为0-based索引
            except:
                elim_week = max_week - 1  # 默认设为最后一周
        else:
            # 其他情况，设为最后一周
            elim_week = max_week - 1

        elimination_info.append(
            {
                "index": i,
                "elim_week": elim_week,
                "name": contestant_names[i],
                "result": result,
            }
        )

    # 按淘汰周次排序，如果淘汰周次相同，按该周评委分数排序（分数低的先淘汰）
    def sort_key(info):
        week = info["elim_week"]
        score = X[info["index"], week] if week < max_week else 0
        return (week, score)

    elimination_info.sort(key=sort_key)

    # 构建淘汰顺序数组e和淘汰周列表
    e = []
    elimination_weeks = []

    for info in elimination_info:
        # 跳过没有被淘汰的选手（如冠军）
        if "1st Place" in info["result"]:
            continue

        e.append(info["index"])
        elimination_weeks.append(info["elim_week"])

    # 转换为numpy数组
    e = np.array(e)
    elimination_weeks = np.array(elimination_weeks)

    # 确定实际需要处理的周数（最大的淘汰周次+1）
    if len(elimination_weeks) > 0:
        T = max(elimination_weeks) + 1
    else:
        T = max_week  # 如果没有淘汰信息，使用最大周数

    # 截断X矩阵到T周
    X = X[:, :T]

    print(f"Season {season_num}: {N} contestants, {T} weeks")
    print(f"Elimination weeks: {elimination_weeks}")
    print(f"Elimination order: {[contestant_names[idx] for idx in e]}")

    return X, e, contestant_names, elimination_weeks


def analyze_season(
    season_num, method="rank", lambda_constraint=1000.0, lambda_smooth=1.0
):
    """分析单个赛季"""
    print(f"\n{'='*60}")
    print(f"Analyzing Season {season_num} using {method} method")
    print("=" * 60)

    try:
        # 准备数据
        X, e, names, elimination_weeks = prepare_season_data(season_num, raw_data)

        if len(e) == 0:
            print(f"Season {season_num} has no elimination data. Skipping...")
            return None

        print(f"X shape: {X.shape}, e length: {len(e)}")

        # 创建估计器
        estimator = FanVoteEstimator(
            method=method,
            lambda_constraint=lambda_constraint,
            lambda_smooth=lambda_smooth,
        )

        # 拟合模型
        estimator.fit(X, e, elimination_weeks=elimination_weeks.tolist())

        # 获取估计的粉丝投票
        V_estimated = estimator.fan_votes_

        # 检查约束满足情况
        print("\nConstraint violations:")
        violations = []
        for week_idx, week in enumerate(elimination_weeks):
            violation = estimator._constraint_violation(week, V_estimated)
            violations.append(violation)
            actual_week = week + 1  # 转换为1-based
            print(f"Week {actual_week}: constraint violation = {violation:.6f}")

        if violations:
            avg_violation = np.mean(violations)
            max_violation = np.max(violations)
            print(f"\nAverage violation: {avg_violation:.6f}")
            print(f"Maximum violation: {max_violation:.6f}")

            # 计算约束满足率
            satisfied = sum(1 for v in violations if v <= estimator.epsilon)
            satisfaction_rate = satisfied / len(violations) * 100
            print(
                f"Constraints satisfied: {satisfied}/{len(violations)} ({satisfaction_rate:.1f}%)"
            )

        # 返回结果
        result = {
            "season": season_num,
            "X": X,
            "e": e,
            "names": names,
            "V": V_estimated,
            "violations": violations,
            "elimination_weeks": elimination_weeks,
            "avg_violation": avg_violation if violations else 0,
            "max_violation": max_violation if violations else 0,
            "satisfaction_rate": satisfaction_rate if violations else 0,
        }

        # 显示部分估计结果
        print(f"\nTop 5 estimated fan votes (first week):")
        first_week_votes = V_estimated[:, 0]
        top_indices = np.argsort(-first_week_votes)[:5]
        for idx in top_indices:
            print(f"  {names[idx]}: {first_week_votes[idx]:.2f}")

        return result

    except Exception as error:
        print(f"Error analyzing season {season_num}: {error}")
        import traceback

        traceback.print_exc()
        return None


def analyze_multiple_seasons(
    seasons, method="rank", lambda_constraint=1000.0, lambda_smooth=1.0
):
    """分析多个赛季"""
    all_results = []

    for season in seasons:
        result = analyze_season(season, method, lambda_constraint, lambda_smooth)
        if result:
            all_results.append(result)

    return all_results


def summarize_results(results):
    """汇总分析结果"""
    if not results:
        print("No results to summarize.")
        return

    print(f"\n{'='*80}")
    print("SUMMARY ACROSS SEASONS")
    print("=" * 80)
    print(
        f"{'Season':<8} {'Contestants':<12} {'Weeks':<8} {'Avg Violation':<15} {'Max Violation':<15} {'Satisfaction':<12}"
    )
    print("-" * 80)

    for res in results:
        season = res["season"]
        contestants = len(res["names"])
        weeks = res["X"].shape[1]
        avg_viol = res["avg_violation"]
        max_viol = res["max_violation"]
        satisfaction = res["satisfaction_rate"]

        print(
            f"{season:<8} {contestants:<12} {weeks:<8} {avg_viol:<15.6f} {max_viol:<15.6f} {satisfaction:<12.1f}%"
        )

    # 计算总体统计
    if results:
        print("-" * 80)
        total_seasons = len(results)
        avg_avg_viol = np.mean([r["avg_violation"] for r in results])
        avg_max_viol = np.mean([r["max_violation"] for r in results])
        avg_satisfaction = np.mean([r["satisfaction_rate"] for r in results])

        print(
            f"{'AVERAGE':<8} {'-':<12} {'-':<8} {avg_avg_viol:<15.6f} {avg_max_viol:<15.6f} {avg_satisfaction:<12.1f}%"
        )
        print(f"\nTotal seasons analyzed: {total_seasons}")

In [42]:
def main():
    """主执行函数"""
    # 加载数据
    global raw_data
    data_path = "2026_MCM_Problem_C_Data.csv"
    raw_data = pd.read_csv(data_path)

    # 显示数据基本信息
    print("=" * 80)
    print("DANCING WITH THE STARS - FAN VOTE ESTIMATION MODEL")
    print("=" * 80)
    print(f"\nData shape: {raw_data.shape}")
    print(f"Seasons available: {sorted(raw_data['season'].unique())}")
    print(f"Total contestants: {len(raw_data)}")

    # 分析所有赛季（或选择特定赛季）
    all_seasons = sorted(raw_data["season"].unique())

    # 为了演示，只分析前10个赛季
    # seasons_to_analyze = all_seasons[:10]
    seasons_to_analyze = all_seasons
    print(f"\nAnalyzing seasons: {seasons_to_analyze}")

    # 设置模型参数
    method = "rank"  # 或 "percent"
    lambda_constraint = 1000.0
    lambda_smooth = 1.0

    # 分析多个赛季
    results = analyze_multiple_seasons(
        seasons=seasons_to_analyze,
        method=method,
        lambda_constraint=lambda_constraint,
        lambda_smooth=lambda_smooth,
    )

    # 汇总结果
    summarize_results(results)

    # 可选：详细分析特定有争议的赛季
    print(f"\n\n{'='*80}")
    print("SPECIAL ANALYSIS: CONTROVERSIAL SEASONS")
    print("=" * 80)

    controversial_seasons = [2, 4, 11, 27]  # 根据问题描述

    for season in controversial_seasons:
        if season in all_seasons:
            print(f"\nAnalyzing controversial Season {season}:")
            result = analyze_season(season, method, lambda_constraint, lambda_smooth)
            if result:
                # 找出有争议的选手
                print(f"\nControversial contestants in Season {season}:")
                for i, name in enumerate(result["names"]):
                    result_str = raw_data[
                        (raw_data["season"] == season)
                        & (raw_data["celebrity_name"] == name)
                    ]["results"].iloc[0]
                    if any(
                        term in str(result_str)
                        for term in ["controvers", "low score", "despite"]
                    ):
                        print(f"  {name}: {result_str}")

    print("\n" + "=" * 80)
    print("ANALYSIS COMPLETE")
    print("=" * 80)


if __name__ == "__main__":
    main()

DANCING WITH THE STARS - FAN VOTE ESTIMATION MODEL

Data shape: (421, 53)
Seasons available: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34)]
Total contestants: 421

Analyzing seasons: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29)

C:\Users\pl050\AppData\Local\Temp\ipykernel_28312\40467324.py:63: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  result = minimize(



Constraint violations:
Week 1: constraint violation = 1.000001
Week 2: constraint violation = 0.000000
Week 3: constraint violation = 5.000001
Week 4: constraint violation = 1.000001
Week 5: constraint violation = 0.000001
Week 6: constraint violation = 1.000001
Week 7: constraint violation = 2.000001
Week 11: constraint violation = 1.000001
Week 11: constraint violation = 1.000001

Average violation: 1.333334
Maximum violation: 5.000001
Constraints satisfied: 2/9 (22.2%)

Top 5 estimated fan votes (first week):
  Master P: 0.86
  Lisa Rinna: 0.84
  Kenny Mayne: 0.83
  Giselle Fernandez: 0.79
  George Hamilton: 0.78

Analyzing Season 3 using rank method
Season 3: 11 contestants, 11 weeks
Elimination weeks: [ 0  1  2  3  4  4  6  7 10 10]
Elimination order: ['Tucker Carlson', 'Shanna Moakler', 'Harry Hamlin', 'Vivica A. Fox', 'Sara Evans', 'Willa Ford', 'Jerry Springer', 'Monique Coleman', 'Joey Lawrence', 'Mario Lopez']
X shape: (11, 11), e length: 10

Constraint violations:
Week 1: c